In [1]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 13198, done.
remote: Total 13198 (delta 0), reused 0 (delta 0), pack-reused 13198
Receiving objects: 100% (13198/13198), 13.12 MiB | 7.53 MiB/s, done.
Resolving deltas: 100% (9061/9061), done.


In [2]:
!pip install -r ./yolov5/requirements.txt

In [3]:
import json
from PIL import Image
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import shutil
import torch

In [4]:
# Create dirs for yolo
!mkdir data_for_yolo
!mkdir data_for_yolo/data
!mkdir data_for_yolo/data/train
!mkdir data_for_yolo/data/val
!mkdir data_for_yolo/data/train/images
!mkdir data_for_yolo/data/train/labels
!mkdir data_for_yolo/data/val/images
!mkdir data_for_yolo/data/val/labels

In [5]:
yaml_content = """
train: ./data_for_yolo/data/train/images/
val: ./data_for_yolo/data/val/images/

# number of classes
nc: 1

# class names
names: ['carplate']
"""

with open('./data_for_yolo/dataset.yaml', 'w') as f:
    f.write(yaml_content)

In [6]:
!mkdir labels

In [7]:
# Transfer dataset to yolo format
image_names = []
with open('../input/carplates/data/train.json', 'r') as f:
    json_file = json.load(f)
    for i in tqdm(range(len(os.listdir('../input/carplates/data/train')[:2000]))):
        try:
            im = Image.open('../input/carplates/data/' + json_file[i]['file'])
        except:
            print('skip image')
            continue
        width_im, height_im = im.size
        for j in range(len(json_file[i]['nums'])):
            bbox = json_file[i]['nums'][0]['box']
            max_x = max([bbox[i][0] for i in range(4)])
            min_x = min([bbox[i][0] for i in range(4)])
            max_y = max([bbox[i][1] for i in range(4)])
            min_y = min([bbox[i][1] for i in range(4)])
            x_center = ((max_x + min_x)/2) / width_im
            y_center = ((max_y + min_y)/2) / height_im
            width = (max_x - min_x) / width_im
            height = (max_y - min_y) / height_im
            file_name = json_file[i]['file'].split('.')[0].split('/')[1]
            image_names.append(json_file[i]['file'].split('/')[1])
            with open(f'./labels/{file_name}.txt', 'a+') as outfile:
                outfile.write(f'0 {x_center} {y_center} {width} {height} \n')

100%|██████████| 2000/2000 [00:12<00:00, 158.54it/s]


In [8]:
len(os.listdir('./labels'))

2000

In [9]:
len(os.listdir('../input/carplates/data/train'))

25633

In [10]:
# Split dataset
train_image_names, val_image_names = train_test_split(image_names, test_size=0.2, random_state=22)

for image_name in tqdm(train_image_names):
    shutil.copy('./labels/' + image_name.split('.')[0] + '.txt',
                './data_for_yolo/data/train/labels/')
    shutil.copy('../input/carplates/data/train/' + image_name,
                './data_for_yolo/data/train/images/')
    
for image_name in tqdm(val_image_names):
    shutil.copy('./labels/' + image_name.split('.')[0] + '.txt',
                './data_for_yolo/data/val/labels/')
    shutil.copy('../input/carplates/data/train/' + image_name,
                './data_for_yolo/data/val/images/')

100%|██████████| 418/418 [00:00<00:00, 688.09it/s]


In [11]:
shutil.move('./data_for_yolo', './yolov5')

'./yolov5/data_for_yolo'

In [14]:
torch.cuda.empty_cache()
!python ./yolov5/train.py --img 1280 --batch 2 --epochs 40 --data ./yolov5/data_for_yolo/dataset.yaml --weights yolov5m6.pt --cfg ./yolov5/models/hub/yolov5m6.yaml --name yolov5m6_results --cache

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=yolov5m6.pt, cfg=./yolov5/models/hub/yolov5m6.yaml, data=./yolov5/data_for_yolo/dataset.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=40, batch_size=2, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=yolov5m6_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-183-gc98128f Python-3.7.12 torch-1.11.0 CUDA:0 (Tesla